# End-To-End Python Client Example

This notebook creates a project named `test`, creates a dummy stream, and writes and reads to it.

In [ ]:
%load_ext autoreload
%autoreload 2
import asyncio
import beneath
import sys
import secrets

Create a client (assumes you have already authenticated with `beneath auth SECRET` on the command-line)

In [ ]:
client = beneath.Client()

Get user and organization info

In [ ]:
me = await client.admin.organizations.find_me()
organization_name = me["name"]
organization_id = me["organizationID"]

Get or create test project

In [ ]:
project_name = "test"
project = await client.admin.projects.stage(organization_name=organization_name, project_name=project_name)

Create test stream

In [ ]:
stream_path = f"{organization_name}/{project_name}/dummies"

In [ ]:
stream = await client.stage_stream(
    stream_path=stream_path,
    schema="""
        type Dummmy @stream() @key(fields: "a") {
            a: String!
            b: Int!
            c: Int
            d: Bytes16
        }
    """,
)

In [ ]:
instance = await stream.stage_instance(version=0, make_primary=True)

Create function for generating random records for the stream

In [ ]:
def generate_record():
    return {
        "a": secrets.token_urlsafe(30),
        "b": secrets.randbelow(sys.maxsize),
        "c": None,
        "d": secrets.token_bytes(16),   
    }

Write records to the stream

In [ ]:
async with instance.writer() as w:
    n = 1000
    records = [generate_record() for _ in range(n)]
    await w.write(records)

Write records forever

In [ ]:
async with instance.writer() as w:
    delay_seconds = 2
    while True:
        record = generate_record()
        await w.write(record)
        await asyncio.sleep(delay_seconds)

Read all records really easily

In [ ]:
df = await beneath.easy_read(stream_path)
df

Read some records with lower-level APIs

In [ ]:
cursor = await instance.query_index()
df = await cursor.read_next(to_dataframe=True)
df

Read all the records with lower-level APIs

In [ ]:
cursor = await instance.query_index()
df = await cursor.read_all(to_dataframe=True)
df

Peek at the latest writes

In [ ]:
cursor = await instance.query_log(peek=True)
df = await cursor.read_next(to_dataframe=True)
df

Write some more and fetch the changes

In [ ]:
cursor = await instance.query_log()

n = 25
records = [generate_record() for _ in range(n)]
async with instance.writer() as w:
    await w.write(records)
    await asyncio.sleep(2)

df = await cursor.read_next_changes(to_dataframe=True)
df

Helper to write records in the background for next demos

In [ ]:
async def write_forever():
    n = 2000
    sleep = 1
    async with instance.writer(write_delay_ms=100) as w:
        while True:
            records = (generate_record() for _ in range(n))
            await w.write(records)
            print(f"Wrote {n} records")
            await asyncio.sleep(sleep)

Write and subscribe to changes with a callback

In [ ]:
async def subscribe_forever():
    async def cb(records, cursor):
        print(f"Received {len(records)} records – Sample: {records[0]['a']}")
    cursor = await instance.query_log()
    await cursor.subscribe_changes(callback=cb, poll_at_most_every_ms=100)

task = asyncio.create_task(write_forever())
try:
    await subscribe_forever()
finally:
    task.cancel()

Write and subscribe to changes with async iterator

In [ ]:
async def iterate_forever():
    cursor = await instance.query_log()
    iterator = cursor.iterate_changes(poll_at_most_every_ms=100)
    async for records in iterator:
        print(f"Received {len(records)} records – Sample: {records[0]['a']}")

task = asyncio.create_task(write_forever())
try:
    await iterate_forever()
finally:
    task.cancel()